# Mecab QnA

## 라이브러리

In [1]:
#pip freeze

In [2]:
import os

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

from tqdm.notebook import tqdm

import re
import pickle

from konlpy.tag import Mecab

## Path

In [3]:
base_path = '../Data/QnA/'
save_path = '../Data/QnA/'

## Load Data

In [4]:
df = pd.read_csv(base_path+'qna_table_hanspell.csv')

In [5]:
df.shape

(95190, 15)

In [6]:
# hanspell 이후 <None> 제거
df = df[(df['question_spellcheck']!='<None>')&(df['answer_spellcheck']!='<None>')]
df.reset_index(drop=True, inplace=True)

In [7]:
print("<None> 제거 후 QnA 갯수: " + str(df.shape))

<None> 제거 후 QnA 갯수: (95189, 15)


In [8]:
print("label별 QnA 갯수 확인")
df['label'].value_counts()

label별 QnA 갯수 확인


상품    45136
배송    30239
교환     8423
환불     5801
기타     3072
반품     2497
Name: label, dtype: int64

## Mecab

In [9]:
mecab = Mecab()

In [10]:
def make_mecab_tokens(comment):

    # mecab 돌리기
    comment_mecab = []
    for i in range(len(comment)):
        if comment[i] == 'NaN': #NaN값 처리ㅣ
            comment_mecab.append('nan')
        else:
            comment_mecab.append(mecab.pos(comment[i]))

    # 조사 제거
    morpheme = ['NNG','NNP','NNB','NNBC','NR','NP','VV','VA','VX','VCP','VCN','MM','MAG','MAJ','IC','SN']    
    tmp = []
    comment_tokens = []
    
    for tok in comment_mecab:
        for t in tok:
            if t[1] in morpheme:
                tmp.append(t[0])
        if len(tmp)==0:
            comment_tokens.append('')
        else:
            comment_tokens.append(tmp)
        tmp = []

    # 한 문장으로 결합
    comment_tokens_str = [' '.join(re) for re in comment_tokens]

    return comment_tokens_str

In [11]:
# question
question = df['question_spellcheck'].astype(str)

df['question_mecab_SN'] = make_mecab_tokens(question)

In [12]:
# answer
answer = df['answer_spellcheck'].astype(str)

df['answer_mecab_SN'] = make_mecab_tokens(answer)

In [13]:
df = df[(df['question_mecab_SN']!='')&(df['answer_mecab_SN']!='')] # NaN 제거
df.reset_index(drop=True, inplace=True)

In [14]:
df.shape

(94955, 17)

In [15]:
df[:1]

,product_id,product_name,product_option,user_id,user_buyer,is_secret,label,question,question_time,answer,answer_time,question_clean,answer_clean,question_spellcheck,answer_spellcheck,question_mecab_SN,answer_mecab_SN
0,388715,순수원목 A사이드테이블 3colors,A사이드테이블 / 우드,3077236,True,False,환불,상부 연결 목재 중 하나가 나사산이 덜 파져 있네요;; 조립하다 목재가 갈라져버립니...,2021-03-18T16:52:34.000+09:00,안녕하세요 고객님 먼데이하우스입니다.\r\n불편을 드려 죄송합니다.\r\n유선상 연...,2021-03-18T18:19:46.000+09:00,상부 연결 목재 중 하나가 나사산이 덜 파져 있네요 조립하다 목재가 갈라져버립니다 ...,안녕하세요 고객님 먼데이하우스입니다 불편을 드려 죄송합니다 유선상 연락을 드렸으나 ...,상부 연결 목재 중 하나가 나사산이 덜 파죠 있네요 조립하다 목재가 갈라져버립니다 ...,안녕하세요 고객님 먼데이 하우스입니다 불편을 드려 죄송합니다 유선상 연락을 드렸으나...,상부 연결 목재 중 하나 나사산 덜 파 있 조립 목재 나사 잘못 각도 것 아니 나사...,안녕 고객 먼데이 하우스 불편 유선 연락 부재중 이 안타깝 조립 시작 반품 어렵 사...


## Labeling

In [16]:
# label 수치형으로 변환
cat = pd.Series(df['label'])
labels, uniques = pd.factorize(cat)
labels, uniques

(array([0, 1, 2, ..., 1, 1, 1]),
 Index(['환불', '상품', '배송', '교환', '기타', '반품'], dtype='object'))

In [17]:
df['num_label'] = labels
df['num_label'] = df['num_label'].astype(str) # str으로 변환

In [18]:
df[['label','num_label']].value_counts()

label  num_label
상품     1            45012
배송     2            30203
교환     3             8384
환불     0             5790
기타     4             3059
반품     5             2486
dtype: int64

In [19]:
#기타와 반품 label 순서 변경
df.loc[df['num_label'] =='4', 'num_label'] = 'tmp'
df.loc[df['num_label'] =='5', 'num_label'] = '4'
df.loc[df['num_label'] =='tmp', 'num_label'] = '5'

In [20]:
df[['label','num_label']].value_counts()

label  num_label
상품     1            45012
배송     2            30203
교환     3             8384
환불     0             5790
기타     5             3059
반품     4             2486
dtype: int64

In [21]:
df.shape

(94955, 18)

In [22]:
# csv 저장 후 내보내기
df.to_csv(save_path+'qna_table_mecab_6_SN.csv', index=False)

## KoBERT용 데이터 생성

* 기타(5) label 제거
* Que + '\t' + Ans + '\t' 형태
* txt 파일로 내보내기

In [23]:
# 기타 제거
df_drop = df[df['label']!='기타']

# Que + '\t' + Ans + '\t'
data_drop = pd.DataFrame()
data_drop['BERT_NEW'] = df_drop['question_mecab_SN'].values + '\t' + df_drop['answer_mecab_SN'].values + '\t' + df_drop['num_label'].astype(str).values + '\n'

data_kb = data_drop['BERT_NEW']
data_train, data_test = train_test_split(data_kb, test_size=0.2, shuffle=True, random_state=316)

# 내보내기
with open(save_path+'type1_SN_BERT_drop_test_new.txt', 'w') as file:
    file.writelines(data_test.values.tolist())
    
with open(save_path+'type1_SN_BERT_drop_train_new.txt', 'w') as file:
    file.writelines(data_train.values.tolist())